# LIGN
Graph Induced Lifelong Learning for Spatial-Temporal Data

----

## Imports

In [ ]:
import lign as lg
import lign.utils.io as io
import torch as th
import torchvision as tv
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

----

## Preprocessing 
The data needs to formatted and set up for lign. A GraphDatabase class is needed to load graphs into the model and track the node. **_Only needs to be ran once_**

### Create Dataset
*_ Only run one of the following cells_ 

*_ Either run "Create Dataset" or "Load Dataset"_

In [ ]:
trans = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])
dataset = io.cifar_to_lign("data/datasets/CIFAR100", transforms = trans)
dataset.save("data/cifar100.lign")

### Load Dataset
*_ Only run one of the following cells_

*_ Either run "Create Dataset" or "Load Dataset"_

In [ ]:
dataset = lg.graph.GraphDataset("data/cifar100.lign")

### Cuda GPUs
Searches for nvidia GPUs in order to speed up training and inference

In [ ]:
device = th.device("cuda" if th.cuda.is_available() else "cpu")

### Functions and NNs
Functions and NNs used throughout the code to apply changes to the graph's data

In [ ]:
def sum_neighs_data(neighs):
    out = neighs[0]
    for neigh in neighs[1:]:
        out = out + neighs
    return out

class G_Linear(nn.Module):
    def __init__(self, in_feats, out_feats, activation):
        super(G_Linear, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats)
        self.activation = activation

    def forward(self, x):
        x = self.linear(x)
        if self.activation:
            x = self.activation(x)
        return x

class G_CNN(nn.Module):
    def __init__(self, in_ch, out_ch, kernel, activation):
        super(G_CNN, self).__init__()
        self.cnn = nn.Conv2d(in_ch, out_ch, kernel)
        self.activation = activation

    def forward(self, x):
        x = self.cnn(x)
        if self.activation:
            x = self.activation(x)
        return x

---

## Model